In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, r2_score, confusion_matrix, roc_curve, auc, precision_recall_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.calibration import calibration_curve
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold # Import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
import pickle

# Load your dataset (update the file path as needed)
df = pd.read_csv('/content/drive/MyDrive/la_hai_la_kei_hola_jasto_xa_hai.csv')

# Convert date column to datetime
df['acq_date'] = pd.to_datetime(df['acq_date'])
df = df.sort_values(by='acq_date')

# **Preprocessing**
print("Missing values per column:")
print(df.isnull().sum())

# Define target variable
df['fire_risk'] = ((df['frp'] > 0).astype(int))

X = df.drop(columns=['fire_risk'])  # Features
y = df['fire_risk']  # Target

rus = RandomUnderSampler(sampling_strategy=0.8, random_state=42)  # 1 fire : 2 no-fire ratio
X_resampled, y_resampled = rus.fit_resample(X, y)

df = X_resampled.assign(fire_risk=y_resampled)

# Separate the majority (0) and minority (1) classes
df_fire = (df['fire_risk'] == 1).sum()
df_no_fire = (df['fire_risk'] == 0).sum()
print(f"Number of fire instances: {df_fire}, Number of no-fire instances: {df_no_fire}")

# Critical new features
df['fuel_moisture_index'] = (df['swvl1_mean'] / 100) * df['t2m_mean']
df['flammability_score'] = (
    df['vpd'] * df['wind_speed'] / (df['swvl4_mean'] + 1e-6)
)
df['persistent_drought'] = df[['7d_drought','14d_drought']].mean(axis=1)

df = df.dropna()

drop_cols = ['latitude', 'longitude', 'fire_count', 'acq_date', 'fire_risk', 'confidence', 'frp', 'brightness','7d_max_d2m', '3d_max_d2m',
             'date', 'cell_id', 'total_frp', 'number', 'latitude_x', 'longitude_x', 't2m_mean', '7d_avg_d2m', '7d_avg_d2m', 'swvl3_mean',
             'nearest_lon', '3d_avg_wind', '7d_avg_wind', '14d_avg_wind', '30d_avg_wind','wind_speed', 'swvl1_mean_7d_avg', 'tp_sum','swvl3_mean_14d_avg',
             'valid_time', 'longitude_y', 'latitude_y', 'wind_temp_interaction','swvl3_mean_7d_avg','swvl2_mean_3d_avg', 'swvl2_mean_7d_avg',
             'date_7d_before', 'nearest_lat', 'geometry', 'centroid_lat', 'centroid_lon', 'grid_id','swvl1_mean_3d_avg', 'swvl1_mean', 'swvl1_mean_14d_avg'
             ]

X = df.drop(columns=drop_cols, errors='ignore')
y = df['fire_risk']

# Random 80-20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Features in X_train (before selection):", X_train.columns.tolist())

# Convert object columns to numeric (essential before scaling or initial model training)
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce').fillna(0)
for col in X_test.columns:
    if X_test[col].dtype == 'object':
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce').fillna(0)

# --- NEW SECTION: Initial Model Training and Feature Selection ---

# We need an initial model to get feature importances.
# Use the same best parameters you found previously for this initial model.
initial_model_params = {
    'scale_pos_weight': 4.5,
    'learning_rate': 0.1,
    'max_depth': 7,
    'subsample': 0.7,
    'colsample_bytree': 0.9,
    'reg_alpha': 1,
    'reg_lambda': 2,
    'gamma': 1,
    'eval_metric': ['auc', 'aucpr', 'logloss'],
    'n_estimators': 600,
    'tree_method': 'hist',
    'early_stopping_rounds': 50,
    'enable_categorical': False,
    'use_label_encoder': False,
    'n_jobs': -1
}
# params = {
#     'scale_pos_weight': 4.5,  # ~4.57
#     'learning_rate': 0.1,
#     'max_depth': 5,
#     'subsample': 0.7,
#     'colsample_bytree': 0.6,
#     'reg_alpha': 0.5,
#     'reg_lambda': 0.8,
#     'gamma': 0.2,
#     'eval_metric': ['auc', 'aucpr', 'logloss']  # Focus on precision-recall AUC
# }

print("\n--- Training initial model for Feature Importance selection ---")
initial_xgb_model = xgb.XGBClassifier(**initial_model_params)

# It's good practice to scale before training even this initial model for importance,
# for consistent preprocessing steps.
temp_scaler = StandardScaler()
X_train_scaled_temp = pd.DataFrame(temp_scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
X_test_scaled_temp = pd.DataFrame(temp_scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
X_train_scaled_temp = np.nan_to_num(X_train_scaled_temp)
X_test_scaled_temp = np.nan_to_num(X_test_scaled_temp)

initial_xgb_model.fit(
    X_train_scaled_temp, y_train,
    eval_set=[(X_test_scaled_temp, y_test)],
    verbose=False
)

# Get feature importances from the initial model
importances = initial_xgb_model.feature_importances_
feature_names = X_train.columns
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)

print("\nTop 10 Feature Importances from Initial Model:")
print(importance_df.head(10))

# Select features based on a threshold (e.g., median importance)
threshold = importance_df['importance'].median()
print(f"\nUsing a feature importance threshold of: {threshold:.4f}")

sfm = SelectFromModel(initial_xgb_model, threshold=threshold, prefit=True)

X_train_selected_df = X_train.loc[:, sfm.get_support()]
X_test_selected_df = X_test.loc[:, sfm.get_support()]

selected_features = X_train_selected_df.columns.tolist()
print(f"\nNumber of selected features: {len(selected_features)}")
print("Selected Features:", selected_features)

# --- END NEW SECTION: Initial Model Training and Feature Selection ---

# Scaling ONLY the selected features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_selected_df), columns=selected_features, index=X_train_selected_df.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_selected_df), columns=selected_features, index=X_test_selected_df.index)

# Handle NaNs after scaling
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = np.nan_to_num(X_test_scaled)

print(f"\nDataset shape: {df.shape}")
print(f"Training set size (selected features): {X_train_scaled.shape}")
print(f"Test set size (selected features): {X_test_scaled.shape}")

# --- NEW SECTION: Hyperparameter Tuning (GridSearchCV) with Selected Features ---
print("\n--- Starting Hyperparameter Tuning (GridSearchCV) on Selected Features ---")

# Define the parameter grid for GridSearchCV
# You can adjust these ranges based on previous insights or desired search space
param_grid = {
    # 'learning_rate': [0.05, 0.1],
    # 'max_depth': [5, 7],
    # 'subsample': [0.7, 0.9],
    # 'colsample_bytree': [0.5, 0.7, 0.9], # Explore this more for overfitting
    # 'reg_alpha': [0.1, 0.5, 1],       # L1 regularization
    # 'reg_lambda': [0.5, 1, 2],       # L2 regularization
    # 'gamma': [0.1, 0.2]
}

# Initialize XGBClassifier with fixed parameters for GridSearchCV
# n_estimators should be set high enough to allow early stopping to find the optimum
# scale_pos_weight is crucial for imbalanced data, so include it here
fire_ratio = len(y_train[y_train == 1]) / len(y_train)
# Adjust scale_pos_weight if needed based on the resampled data's ratio
# In this case, your `rus` balances, so a specific scale_pos_weight might still be beneficial if it's not 1:1.
# Based on previous best, keeping 4.5.
fixed_params = {
    'objective': 'binary:logistic',
    'n_estimators': 1000, # Set a high number for early stopping
    'tree_method': 'hist',
    'enable_categorical': False,
    'use_label_encoder': False,
    'n_jobs': -1,
    'random_state': 42,
    'scale_pos_weight': 4.5 # Keep this as it's a critical imbalance parameter
}

# For GridSearchCV, we often don't use early_stopping directly in the estimator,
# but rather through `callbacks` or let `fit` handle it if it supports it within CV,
# or simply choose a high `n_estimators` and rely on CV to pick robust parameters.
# For simplicity, we'll let GridSearchCV find the best params and then train with early stopping.

xgb_base = xgb.XGBClassifier(**fixed_params)

# Setup StratifiedKFold for cross-validation to maintain class balance
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='roc_auc', # Optimize for ROC AUC
    cv=cv,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train)

print("\n--- Hyperparameter Tuning Complete ---")
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best ROC AUC score: {grid_search.best_score_:.4f}")

best_params = grid_search.best_params_

# --- END NEW SECTION: Hyperparameter Tuning (GridSearchCV) ---


# **XGBoost Model with Early Stopping and Evaluation History (TRAINING WITH BEST HYPERPARAMS)**
evals_result = {}

# Combine fixed_params with best_params from GridSearchCV
final_model_params = {**fixed_params, **best_params}
# Ensure eval_metric is passed correctly
final_model_params['eval_metric'] = ['auc', 'aucpr', 'logloss']
final_model_params['early_stopping_rounds'] = 50 # Re-add early stopping for final fit

model = xgb.XGBClassifier(**final_model_params)

print("\n--- Training final model with BEST selected features and BEST hyperparameters ---")
model.fit(
    X_train_scaled, y_train,
    eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)],
    verbose=1 # Keep verbose to see training progress
)

evals_result = model.evals_result()

# **Training Progress Visualization**
def plot_training_progress(evals_result):
    """Plot training progress for all metrics"""
    eval_sets = list(evals_result.keys())
    if len(eval_sets) == 0:
        print("No evaluation results found. Make sure eval_set is provided during training.")
        return

    metrics = list(evals_result[eval_sets[0]].keys())
    n_metrics = len(metrics)

    fig, axes = plt.subplots(1, n_metrics, figsize=(n_metrics * 6, 5))
    if n_metrics == 1:
        axes = [axes]

    colors = ['blue', 'red']
    eval_names = ['Train', 'Test']

    for i, metric in enumerate(metrics):
        ax = axes[i]

        for j, eval_set in enumerate(eval_sets):
            if j < len(colors):
                scores = evals_result[eval_set][metric]
                epochs = range(1, len(scores) + 1)
                ax.plot(epochs, scores, label=f'{eval_names[j]} {metric.upper()}',
                        color=colors[j], linewidth=2, alpha=0.8)

        ax.set_xlabel('Boosting Rounds (Iterations)', fontweight='bold')
        ax.set_ylabel(metric.upper(), fontweight='bold')
        ax.set_title(f'{metric.upper()} Progress', fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)

        if len(eval_sets) > 1:
            test_scores = evals_result[eval_sets[1]][metric]
            if metric == 'logloss':
                best_iter = np.argmin(test_scores)
            else:
                best_iter = np.argmax(test_scores)
            ax.axvline(x=best_iter+1, color='green', linestyle='--',
                       label=f'Best iteration: {best_iter+1}')
            ax.legend()

    plt.tight_layout()
    plt.show()

# Plot training progress
print("\n=== Training Progress Visualization (Best Hyperparameters) ===")
plot_training_progress(evals_result)


# **Additional Training Analysis**
def plot_detailed_training_analysis(evals_result):
    """Plot detailed analysis of training progress"""

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    if 'auc' in evals_result['validation_0']:
        train_auc = evals_result['validation_0']['auc']
        test_auc = evals_result['validation_1']['auc']
        epochs = range(1, len(train_auc) + 1)

        axes[0].plot(epochs, train_auc, label='Train AUC', color='blue', linewidth=2)
        axes[0].plot(epochs, test_auc, label='Test AUC', color='red', linewidth=2)
        axes[0].fill_between(epochs, train_auc, test_auc, alpha=0.2, color='gray')
        axes[0].set_xlabel('Boosting Rounds', fontweight='bold')
        axes[0].set_ylabel('AUC Score', fontweight='bold')
        axes[0].set_title('AUC Learning Curve', fontweight='bold')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

    if 'aucpr' in evals_result['validation_0']:
        train_prauc = evals_result['validation_0']['aucpr']
        test_prauc = evals_result['validation_1']['aucpr']
        epochs = range(1, len(train_prauc) + 1)

        axes[1].plot(epochs, train_prauc, label='Train PR-AUC', color='blue', linewidth=2)
        axes[1].plot(epochs, test_prauc, label='Test PR-AUC', color='red', linewidth=2)
        axes[1].fill_between(epochs, train_prauc, test_prauc, alpha=0.2, color='gray')
        axes[1].set_xlabel('Boosting Rounds', fontweight='bold')
        axes[1].set_ylabel('PR-AUC Score', fontweight='bold')
        axes[1].set_title('Precision-Recall AUC Learning Curve', fontweight='bold')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

    if 'logloss' in evals_result['validation_0']:
        train_loss = evals_result['validation_0']['logloss']
        test_loss = evals_result['validation_1']['logloss']
        epochs = range(1, len(train_loss) + 1)

        axes[2].plot(epochs, train_loss, label='Train Log Loss', color='blue', linewidth=2)
        axes[2].plot(epochs, test_loss, label='Test Log Loss', color='red', linewidth=2)
        axes[2].fill_between(epochs, train_loss, test_loss, alpha=0.2, color='gray')
        axes[2].set_xlabel('Boosting Rounds', fontweight='bold')
        axes[2].set_ylabel('Log Loss', fontweight='bold')
        axes[2].set_title('Log Loss Learning Curve', fontweight='bold')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\n=== Overfitting Analysis (Best Hyperparameters) ===")
    if 'auc' in evals_result['validation_0']:
        final_train_auc = evals_result['validation_0']['auc'][-1]
        final_test_auc = evals_result['validation_1']['auc'][-1]
        auc_gap = final_train_auc - final_test_auc
        print(f"Final Train AUC: {final_train_auc:.4f}")
        print(f"Final Test AUC: {final_test_auc:.4f}")
        print(f"AUC Gap (Train - Test): {auc_gap:.4f}")

        if auc_gap > 0.05:
            print("⚠️  Potential overfitting detected (AUC gap > 0.05)")
        else:
            print("✅ No significant overfitting detected")

    print(f"\n=== Early Stopping Analysis (Best Hyperparameters) ===")
    print(f"Total boosting rounds completed: {model.n_estimators}")
    print(f"Best iteration: {model.best_iteration}")
    print(f"Best score: {model.best_score:.4f}")

# Plot detailed training analysis
plot_detailed_training_analysis(evals_result)

# Set feature names explicitly for the final model (using selected features)
model.get_booster().feature_names = selected_features
print("Feature names set in model (after selection):", model.get_booster().feature_names)

# **Predictions**
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# Find optimal threshold using precision-recall tradeoff
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
y_pred_optimized = ((y_proba >= optimal_threshold)).astype(int)
print(f"\nOptimal threshold (based on F1-score optimization): {optimal_threshold:.4f}")

# **Evaluation Metrics**
print("\n=== Model Performance Metrics (Best Hyperparameters) ===")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_optimized))
print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")
print(f"R² Score: {r2_score(y_test, y_proba):.4f}")
mae = mean_absolute_error(y_test, y_pred_optimized)
mse = mean_squared_error(y_test, y_pred_optimized)
rmse = np.sqrt(mse)
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")

# **Visualizations**

## Confusion Matrix
cm = confusion_matrix(y_test, y_pred_optimized)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title("Confusion Matrix (Best Hyperparameters)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontweight='bold')
plt.ylabel('True Positive Rate', fontweight='bold')
plt.title('ROC Curve (Best Hyperparameters)', fontweight='bold')
plt.legend(loc="lower right")
plt.show()

## Precision-Recall Curve
precision, recall, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(6, 4))
plt.plot(recall, precision, color='b', lw=2)
plt.xlabel('Recall', fontweight='bold')
plt.ylabel('Precision', fontweight='bold')
plt.title('Precision-Recall Curve (Best Hyperparameters)', fontweight='bold')
plt.show()

## Calibration Curve
prob_true, prob_pred = calibration_curve(y_test, y_proba, n_bins=10)
plt.figure(figsize=(6, 4))
plt.plot(prob_pred, prob_true, marker='o', label='Calibration Curve')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Mean Predicted Probability', fontweight='bold')
plt.ylabel('Fraction of Positives', fontweight='bold')
plt.title('Calibration Curve (Best Hyperparameters)', fontweight='bold')
plt.legend()
plt.show()

## Feature Importance (for the final model with selected features)
importances_final = model.feature_importances_
feature_names_final = selected_features
importance_df_final = pd.DataFrame({'feature': feature_names_final, 'importance': importances_final})
importance_df_final = importance_df_final.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=importance_df_final.head(len(selected_features)))
plt.title('Feature Importances (Final Model with Best Hyperparameters)', fontweight='bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')
plt.xlabel('Importance', fontweight='bold')
plt.ylabel('Feature', fontweight='bold')
plt.show()


# **Training Summary**
print("\n=== Training Summary (Best Hyperparameters) ===")
print(f"✅ Model training completed successfully with selected features and best hyperparameters!")
print(f"📊 Best iteration: {model.best_iteration}")
print(f"🎯 Best validation score: {model.best_score:.4f}")
print(f"🔄 Total boosting rounds: {model.n_estimators}")
print(f"⏱️  Early stopping triggered at round: {model.best_iteration}")

# Save the model and scaler (updated for selected features and best hyperparameters)
# import pickle
# with open('xgboost_model_final.pkl', 'wb') as f:
#     pickle.dump(model, f)
# with open('scaler_final.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# --- Your Model Performance Data ---
# Extracting the scores directly from your provided output
train_auc = 0.9995
test_auc = 0.9432

# From Classification Report (macro avg for balanced view across classes)
accuracy = 0.88
macro_precision = 0.88
macro_recall = 0.88
macro_f1 = 0.88

# Other metrics
roc_auc_score_val = 0.9430 # This is essentially the same as test_auc, will use test_auc for consistency
r2_score_val = 0.5971
mae = 0.1211
mse = 0.1211
rmse = 0.3480

# --- Create DataFrames for Plotting ---

# 1. Classification Metrics
classification_metrics = {
    'Metric': ['Train AUC', 'Test AUC', 'Accuracy', 'Macro Precision', 'Macro Recall', 'Macro F1-Score'],
    'Score': [train_auc, test_auc, accuracy, macro_precision, macro_recall, macro_f1]
}
df_classification = pd.DataFrame(classification_metrics)

# 2. Regression/Error Metrics
regression_metrics = {
    'Metric': ['R² Score', 'MAE', 'MSE', 'RMSE'],
    'Score': [r2_score_val, mae, mse, rmse]
}
df_regression = pd.DataFrame(regression_metrics)

# --- Plotting the Bar Charts ---

# Plot 1: Classification Metrics
plt.figure(figsize=(12, 7))
sns.barplot(x='Score', y='Metric', data=df_classification, palette='viridis')
plt.xlim(0, 1.0) # Classification metrics are typically between 0 and 1
plt.title('Key Classification Performance Metrics', fontsize=16, fontweight='bold')
plt.xlabel('Score', fontsize=12, fontweight='bold')
plt.ylabel('Metric', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Add score values on the bars
for index, row in df_classification.iterrows():
    plt.text(row.Score + 0.01, index, f'{row.Score:.4f}', color='black', ha="left", va='center', fontsize=10)

plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Plot 2: Regression/Error Metrics
plt.figure(figsize=(10, 6))
sns.barplot(x='Score', y='Metric', data=df_regression, palette='magma')
plt.title('Regression/Error Performance Metrics', fontsize=16, fontweight='bold')
plt.xlabel('Score', fontsize=12, fontweight='bold')
plt.ylabel('Metric', fontsize=12, fontweight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Add score values on the bars
for index, row in df_regression.iterrows():
    plt.text(row.Score + (0.01 if row.Score < 0.5 else -0.05), index, f'{row.Score:.4f}',
             color='black' if row.Score < 0.5 else 'white', ha="left" if row.Score < 0.5 else "right", va='center', fontsize=10)

plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()